In [11]:
import os
import librosa
import numpy as np
import pandas as pd
import soundfile as sf  # For handling FLAC & WAV files
import random



from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras import backend as K

# Suppress warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

TensorFlow version:  2.18.0


In [12]:
# TIME INTENSIVE


# Define correct paths
#base_dir = "./AVSSpoof/LA/LA/ASVspoof2019_LA_train/flac"  # Already points to 'flac'
base_dir = "./AVSSpoof/LA/LA/ASVspoof2019_LA_dev/flac" 
dataset_dir = "./AVSSpoof/LA/LA"
#protocol_file = os.path.join(dataset_dir, "ASVspoof2019_LA_cm_protocols", "ASVspoof2019.LA.cm.train.trn.txt")
protocol_file = os.path.join(dataset_dir, "ASVspoof2019_LA_cm_protocols", "ASVspoof2019.LA.cm.dev.trl.txt")

# Ensure file exists before running
if not os.path.exists(protocol_file):
    raise FileNotFoundError(f"Protocol file not found: {protocol_file}")

# Feature extraction functions
def extract_mfcc(file_path, n_mfcc=128):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    #print(mfcc.shape)
    return np.mean(mfcc.T, axis=0)  # Mean aggregation over time

def extract_mfsc(file_path, n_mels=128, n_fft=2048, hop_length=512):
    y, sr = librosa.load(file_path, sr=None)
    mfsc = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    #print(mfsc.shape)
    return np.mean(mfsc.T, axis=0)  # Mean aggregation over time

# Load metadata and extract features
def load_data(protocol_file, base_dir):
    data = []
    with open(protocol_file, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            
            # Ensure valid tokens before proceeding
            if len(tokens) < 2:
                print(f"Skipping malformed line: {line.strip()}")
                continue

            file_id, label = tokens[1], tokens[-1]

            # FIX: Do NOT add 'flac' again since base_dir already includes it
            file_path = os.path.join(base_dir, file_id + ".flac")

            # Check if the file actually exists
            if not os.path.exists(file_path):
                print(f"Warning: File not found - {file_path}")
                continue

            features = extract_mfcc(file_path)
            data.append((features, label))

    return pd.DataFrame(data, columns=["features", "label"])

# Load training data
train_data = load_data(protocol_file, base_dir)

# Print results
print(train_data.head())

                                            features     label
0  [-337.1201, 61.995434, -2.45123, 19.191284, 1....  bonafide
1  [-342.31143, 44.619663, -6.771269, 9.524557, -...  bonafide
2  [-369.46967, 43.674416, -1.1685817, 26.372791,...  bonafide
3  [-296.1914, 46.492588, -5.0006576, 19.09601, -...  bonafide
4  [-328.49338, 60.683876, 1.5566636, 15.033571, ...  bonafide


---
title: "Metrics for Binary Classification"
output: html_document
---

## Precision
Measures how many of the positively predicted cases were actually positive.

**Related to Type 1 Error (False Positives):**  
A lower precision indicates a higher Type 1 error rate.

---

## Recall (Sensitivity)
Measures how many actual positives were correctly predicted.

**Related to Type 2 Error (False Negatives):**  
A lower recall indicates a higher Type 2 error rate.

---

## Specificity
Measures how many actual negatives were correctly predicted.


**Relationship to Type 1 Error:**  
Type 1 error is directly measured as:


---



In [13]:
# Prepare features and labels
y = []

X = np.array(train_data["features"].tolist())
y = LabelEncoder().fit_transform(train_data["label"])
print("x Shape:", X.shape) 
print("y Shape:", y.shape)  

# Random 80/20 split
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Cross Validation
x_array = X
y_array = np.array(y)

print("x_array Shape:", x_array.shape) 
print("y_array Shape:", y_array.shape)  

x Shape: (24844, 128)
y Shape: (24844,)
x_array Shape: (24844, 128)
y_array Shape: (24844,)


In [14]:
# Set the seed for numpy (used in data preprocessing)
np.random.seed(42)

# Set the seed for Python's built-in random module (used in shuffling, etc.)
random.seed(42)

# Set the TensorFlow seed
tf.random.set_seed(42)

# Ensure deterministic operations in TensorFlow (useful for GPU operations)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED'] = '42'

In [15]:
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
           
accuracy_scores = []
precision_scores = []
recall_scores = []
auc_scores = []   

results = []
best_auc = 0
best_model = None
best_settings = {}

early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)  # Reduce from default 0.001

# Keras Log Capture
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# CROSS VALIDATION | K FOLDS
for fold, (train_idx, val_idx) in enumerate(kf.split(x_array)):
    print(f"Training Fold {fold+1}/{k}...")

    x_train, x_val = x_array[train_idx], x_array[val_idx]
    y_train, y_val = y_array[train_idx], y_array[val_idx]

    smote = SMOTE(random_state=42)
    x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

    # Build the model
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(x_train.shape[1],)),  # Define input shape here
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.1), # ORG .3
        tf.keras.layers.Dense(64, activation='relu'),
        #tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 
    #model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 
                           Precision(name='precision'), 
                           Recall(name='recall'), 
                           AUC(curve='PR', name='auc_pr')])  # PR AUC is related to F1 Score

    #history = model.fit(x_train, y_train, validation_data=(x_val, y_val),verbose=0, epochs=100, batch_size=32, callbacks=[early_stopping])
    history = model.fit(x_train_resampled, y_train_resampled, validation_data=(x_val, y_val),verbose=0, epochs=100, batch_size=32, callbacks=[early_stopping])

    scores = model.evaluate(x_val, y_val, verbose=0)
    accuracy, precision, recall, auc = scores[1],scores[2],scores[3],scores[4]

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    auc_scores.append(auc)
    #Compute average metrics
    print("\n **Final Cross-Validation Results:**")
    #print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
    #print(f"Average Precision: {np.mean(precision_scores):.4f}")
    #print(f"Average Recall: {np.mean(recall_scores):.4f}")
    #print(f"Average AUC: {np.mean(auc_scores):.4f}")
    #print("-------------------------------------------")

    # SAVE BEST MODEL
    results.append({
        "fold": fold+1,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "auc": auc})
    print(f"Fold {fold+1} | Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUC: {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_settings = {            
            "fold": fold+1,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "auc": auc,
            "layers":[200,128,64],
            "epochs": 50,
            "batch_size": 32,
            "optimizer": "adam"}
    

Training Fold 1/10...


2025-02-06 18:33:43.148126: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
2025-02-06 18:34:47.138904: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),


 **Final Cross-Validation Results:**
Fold 1 | Accuracy: 0.9992, Precision: 0.9996, Recall: 0.9996, AUC: 1.0000
Training Fold 2/10...


2025-02-06 18:37:54.485996: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}



 **Final Cross-Validation Results:**
Fold 2 | Accuracy: 0.9988, Precision: 0.9987, Recall: 1.0000, AUC: 1.0000
Training Fold 3/10...


...

2025-02-06 18:50:52.182444: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}



 **Final Cross-Validation Results:**
Fold 8 | Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, AUC: 1.0000
Training Fold 9/10...


2025-02-06 18:52:24.507479: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}



 **Final Cross-Validation Results:**
Fold 9 | Accuracy: 0.9891, Precision: 1.0000, Recall: 0.9879, AUC: 1.0000
Training Fold 10/10...

 **Final Cross-Validation Results:**
Fold 10 | Accuracy: 0.9996, Precision: 0.9996, Recall: 1.0000, AUC: 1.0000


2025-02-06 18:53:23.133835: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


In [16]:
#history = model.fit(X_train, y_train, validation_data=(X_val, y_val),verbose=0, epochs=500, batch_size=32, callbacks=[early_stopping])

In [17]:
# Print best validation accuracy and loss
#best_epoch = np.argmin(history.history['loss'])  # Get epoch with lowest validation loss
#best_val_loss = history.history['loss'][best_epoch]
#best_val_acc = history.history['accuracy'][best_epoch]
#best_precision = history.history['precision'][best_epoch]
#best_recall = history.history['recall'][best_epoch]
#best_auc = history.history['auc_pr'][best_epoch]

#print(f"Best Epoch: {best_epoch+1}")  # +1 because epochs start from 0
#print(f"Best Validation Loss: {best_val_loss:.4f}")
#print(f"Best Validation Accuracy: {best_val_acc:.4f}")
#print(f"Best Precision: {best_precision:.4f}")
#print(f"Best Recall: {best_recall:.4f}")
#print(f"Best AUC: {best_auc:.4f}")

In [18]:
avg_results = {
    "avg_accuracy": np.mean([r["accuracy"] for r in results]),
    "avg_precision": np.mean([r["precision"] for r in results]),
    "avg_recall": np.mean([r["recall"] for r in results]),
    "avg_auc": np.mean([r["auc"] for r in results])
}

# MFCC
Best Epoch: 39
Best Validation Loss: 0.0088
Best Validation Accuracy: 0.9969
Best Precision: 0.9982
Best Recall: 0.9984
Best AUC: 0.9999

# MFSC
Best Epoch: 85
Best Validation Loss: 0.1235
Best Validation Accuracy: 0.9512
Best Precision: 0.9635
Best Recall: 0.9829
Best AUC: 0.9960

In [19]:
# Assuming 'model' is your trained model
#model.save("./AVSSpoof/audio_spoofing_model.keras")

# Save best model
best_model.save("./AVSSpoof/audio_spoofing_model.keras")

# Print final results
print("\n **Final Cross-Validation Results:**")
print(f"Best Model from Fold {best_settings['fold']}: AUC = {best_auc:.4f}")
print(f"Average Accuracy: {avg_results['avg_accuracy']:.4f}")
print(f"Average Precision: {avg_results['avg_precision']:.4f}")
print(f"Average Recall: {avg_results['avg_recall']:.4f}")
print(f"Average AUC: {avg_results['avg_auc']:.4f}")
print("\nBest Model Settings:", best_settings)




 **Final Cross-Validation Results:**
Best Model from Fold 4: AUC = 1.0000
Average Accuracy: 0.9981
Average Precision: 0.9996
Average Recall: 0.9983
Average AUC: 1.0000

Best Model Settings: {'fold': 4, 'accuracy': 0.9991951584815979, 'precision': 0.9991079568862915, 'recall': 1.0, 'auc': 1.0, 'layers': [200, 128, 64], 'epochs': 50, 'batch_size': 32, 'optimizer': 'adam'}


In [20]:
# Clear any logs from previous runs
%rm -rf logs/scalars/*